In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1.0,
    max_tokens=100,
    n=1,
    stop="."
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(


In [2]:
from agential.cog.lats.strategies.qa import LATSHotQAStrategy
from agential.cog.lats.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT, LATS_INSTRUCTION_HOTPOTQA, LATS_REFLECT_INSTRUCTION_HOTPOTQA
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
strategy = LATSHotQAStrategy(llm=llm)


In [3]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'
key = "Badr Hari"

root = strategy.initialize()

children_nodes = strategy.generate(
    node=root,
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
